# imports

In [1]:
import os
import glob
import sys
import math

import numpy as np
import tensorflow as tf

# import - custom

In [2]:
sys.path.append("/home/chy/dev-bench/auto-catch-bak/")

In [3]:
%run ./item_index.py

# features

In [23]:
def wrap_to_list(value):
    if not isinstance(value, list):
        return [value]
    return value

def feature_int64(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=wrap_to_list(value)))

def feature_bytes(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=wrap_to_list(value)))

def feature_float(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=wrap_to_list(value)))

In [51]:
def load_npz_from(data_path, block_id):
    npz_path = os.path.join(data_path, "item-{:05d}.npz".format(block_id))
    return load_npz(npz_path)
    
def load_npz(npz_path):
    npz = np.load(npz_path)
    images = npz['images']
    indexes = npz['metas']
    return ItemIndex.from_npz(indexes, images)

def to_tfexample(item_index):
    cx = item_index.camera_x.astype(np.float)
    cy = item_index.camera_y.astype(np.float)
    iff = item_index.image_frame
    ile = item_index.image_left_eye
    ire = item_index.image_right_eye
    
    # NOTE: 이미지는 모두 jpeg encoded 상태
    features = tf.train.Features(feature={
        'image/frame': feature_bytes(iff.tostring()),     
        'image/left_eye': feature_bytes(iff.tostring()),  
        'image/right_eye': feature_bytes(ire.tostring()), 
        'label/cam_x': feature_float(cx),           
        'label/cam_y': feature_float(cy),           
    })
    
    example = tf.train.Example(features=features)
    return example

def write_tfexample(item_indexs, writer, block_path):
    for item_index in item_indexs:
        try:
            example = to_tfexample(item_index)
            writer.write(example.SerializeToString())
        except Exception as e:
            print('UID [{}] exception: {}'.format(item_index.uid, e))
    print("converted from: {}".format(block_path))
        
def make_writer(out_path, idx):
    options = tf.python_io.TFRecordOptions(tf.python_io.TFRecordCompressionType.GZIP)
    path = os.path.join(out_path, 'everyone_{:05d}.tfr'.format(idx))
    writer = tf.python_io.TFRecordWriter(path=path, options=options)
    return writer
    
def convert(out_path, npz_path):
    print("*** convert ***")
    if not os.path.exists(out_path):
        os.makedirs(out_path)
    blocks = glob.glob(os.path.join(npz_path, "item-*.npz"))
    writer, writer_idx = None, 0
    for i, block_path in enumerate(blocks):
        if i % 16 == 0:
            if writer is not None:
                writer.close()
            writer = make_writer(out_path, writer_idx)
            writer_idx += 1
        item_indexs = load_npz(block_path)
        write_tfexample(item_indexs, writer, block_path)
    writer.close()
    
    print("*** complete ***")

# main

In [6]:
# ls S:datasets\processed\everyone-tfr

In [7]:
# ls I:\dev-visual\enas-origin\src\dataset\item_index.py

In [8]:
# item_index = load_npz("S:datasets\processed\everyone", 1)

In [53]:
npz_path = "/home/chy/archive-data/processed/everyone-npz"
out_path = "/home/chy/archive-data/processed/everyone-tfr"
convert(out_path, npz_path)

*** convert ***
converted from: /home/chy/archive-data/processed/everyone-npz/item-00973.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-00869.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-01710.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-02343.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-01814.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-02152.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-00572.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-02469.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-01151.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-01389.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-00843.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-01722.npz
converted from: /home/chy/archive-data/processed/everyone-np

converted from: /home/chy/archive-data/processed/everyone-npz/item-00235.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-02760.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-01589.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-00161.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-01418.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-02516.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-02185.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-00465.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-01080.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-00377.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-02727.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-00279.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-01781.npz

converted from: /home/chy/archive-data/processed/everyone-npz/item-00737.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-01490.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-00293.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-02424.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-00405.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-00833.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-00998.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-01144.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-02806.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-02239.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-00632.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-00662.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-00824.npz

converted from: /home/chy/archive-data/processed/everyone-npz/item-02464.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-00132.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-01575.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-02758.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-02177.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-00912.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-01274.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-01717.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-01450.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-00530.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-01774.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-00922.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-02674.npz

converted from: /home/chy/archive-data/processed/everyone-npz/item-01633.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-02724.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-02473.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-00337.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-00064.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-00346.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-00176.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-01316.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-02337.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-00488.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-02223.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-02284.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-00900.npz

...

converted from: /home/chy/archive-data/processed/everyone-npz/item-01640.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-00571.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-02670.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-01725.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-02723.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-01479.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-00732.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-01806.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-01832.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-02614.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-01297.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-01150.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-01992.npz

converted from: /home/chy/archive-data/processed/everyone-npz/item-00062.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-01970.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-02203.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-01808.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-00273.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-02354.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-01404.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-00227.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-00766.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-01658.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-01442.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-02528.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-02597.npz

converted from: /home/chy/archive-data/processed/everyone-npz/item-00797.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-02604.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-02623.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-01650.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-01430.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-00060.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-02063.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-02388.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-00916.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-02100.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-01283.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-00327.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-02808.npz

converted from: /home/chy/archive-data/processed/everyone-npz/item-02082.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-02482.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-01773.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-00507.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-00368.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-01291.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-00867.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-00319.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-00101.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-00792.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-01367.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-01063.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-00109.npz

converted from: /home/chy/archive-data/processed/everyone-npz/item-02197.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-02085.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-01265.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-00430.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-00414.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-01705.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-02505.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-00773.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-00558.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-01043.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-00847.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-00002.npz
converted from: /home/chy/archive-data/processed/everyone-npz/item-02066.npz